# 三 深度学习计算

## 1 层和块

块（block）可以描述单个层、由多个层组成的组件或整个模型本身

### 1.1 自定义块

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

net = MLP()
X = torch.rand(2, 20)
print(net(X))

tensor([[ 0.0128, -0.0025,  0.0374,  0.3120, -0.1332, -0.0044,  0.0997,  0.2114,
         -0.0620,  0.2171],
        [ 0.0310, -0.0027, -0.0140,  0.2202, -0.0301,  0.0031,  0.1007,  0.1734,
          0.0482,  0.1303]], grad_fn=<AddmmBackward0>)


### 1.2 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1835, -0.2032, -0.0392,  0.0605, -0.0599, -0.1627, -0.0272,  0.2135,
         -0.0502, -0.0136],
        [ 0.1679, -0.1983, -0.1034, -0.0154, -0.0070, -0.0652, -0.1066,  0.1654,
         -0.1129, -0.0603]], grad_fn=<AddmmBackward0>)

### 1.3 在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
X = torch.rand(2, 20)
net(X)

tensor(0.0322, grad_fn=<SumBackward0>)

### 1.4 混合搭配组合块

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
X = torch.rand(2, 20)
chimera(X)

tensor(-0.1181, grad_fn=<SumBackward0>)

## 2 参数管理

In [10]:
# MLP
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0159],
        [-0.0702]], grad_fn=<AddmmBackward0>)

### 2.1 参数访问

In [15]:
# 参数访问
print(net[2].state_dict())

# 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None)

# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
print(net.state_dict()['2.bias'].data)

# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print("rgnet(X):", rgnet(X))
print(rgnet)
print(rgnet[0][1][0].bias.data)

OrderedDict([('weight', tensor([[-0.1115,  0.3332, -0.2917, -0.1062, -0.1717, -0.0181,  0.2322,  0.3126]])), ('bias', tensor([-0.1859]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1859], requires_grad=True)
tensor([-0.1859])
True
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([-0.1859])
rgnet(X): tensor([[-0.4651],
        [-0.4651]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4

### 2.2 参数初始化

#### 2.2.1 内置初始化

In [16]:
# 所有权重参数初始化为标准差为0.01的高斯随机变量，且将偏置参数设置为0
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

# 将所有参数初始化为给定的常数，比如初始化为1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

# 对某些块应用不同的初始化方法
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5456,  0.6682, -0.1151, -0.0603])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 2.2.2 自定义初始化

In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
print(net[0].weight[:2])

net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[-6.0905, -5.5572,  5.4603, -5.2197],
        [ 0.0000,  7.9508, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)
tensor([42.0000, -4.5572,  6.4603, -4.2197])


### 2.3 参数绑定

In [19]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 3 自定义层

### 3.1 不带参数的层

In [20]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(7.4506e-09, grad_fn=<MeanBackward0>)

### 3.2 带参数的层

In [21]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3)
print(linear.weight)
print(linear(torch.rand(2, 5)))

net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
print(net(torch.rand(2, 64)))

Parameter containing:
tensor([[ 1.6483, -0.2844, -0.6765],
        [ 0.6821, -1.7827,  0.1101],
        [ 0.0689, -1.0431,  1.2657],
        [-0.7529, -1.1837,  0.5303],
        [-0.1335, -0.4410, -1.0872]], requires_grad=True)
tensor([[2.5025, 0.0000, 0.0000],
        [2.9372, 0.0000, 0.0000]])
tensor([[0.],
        [0.]])


## 4 读写文件

### 4.1 加载和保存张量

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

# x = torch.arange(4)
# # 保存
# torch.save(x, 'x-file')
# # 读取
# x2 = torch.load('x-file')

# y = torch.zeros(4)
# torch.save([x, y],'x-files')
# x2, y2 = torch.load('x-files')

# mydict = {'x': x, 'y': y}
# torch.save(mydict, 'mydict')
# mydict2 = torch.load('mydict')

### 4.2 加载和保存模型参数

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

# # 保存模型参数
# torch.save(net.state_dict(), 'mlp.params')

# # 加载模型参数
# clone = MLP()
# clone.load_state_dict(torch.load('mlp.params'))
# clone.eval()

# # 验证
# Y_clone = clone(X)
# Y_clone == Y

## 5 GPU

In [22]:
import torch
from torch import nn

def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu().

    Defined in :numref:`sec_use_gpu`"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    try:
        return torch.device('mps')
    except:
        return torch.device('cpu')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists.

    Defined in :numref:`sec_use_gpu`"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    try:
        device_macos = torch.device('mps')
    except:
        device_macos = torch.device('cpu')
    return devices if devices else [device_macos]

print(torch.device('cpu'))
print(torch.device('cuda'))
print(torch.device('cuda:1'))
print(torch.cuda.device_count())
print(try_gpu())
print(try_gpu(10))
print(try_all_gpus())

X = torch.ones(2, 3, device=try_gpu())
print(X)

net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
print(net(X))

print(net[0].weight.data.device)

cpu
cuda
cuda:1
0
mps
mps
[device(type='mps')]
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')
tensor([[0.5631],
        [0.5631]], device='mps:0', grad_fn=<LinearBackward0>)
mps:0
